Cleaning up the txt file for the tool versions and merging it with the json configs.

In [69]:
import json
import polars as pl
from pathlib import Path
import re
from typing import Dict, List, Union

In [70]:
tools= [tool for tool in Path("../tool_configs/").glob(pattern="./*json")]
bla=[]
for tool in tools:
    tool_dict = json.load(open(tool))
    tool_df = pl.DataFrame(tool_dict)
    bla.append(tool_df)
all_tool_df= pl.concat(bla,how="diagonal_relaxed")
all_tool_df = all_tool_df.with_columns(
    pl.col("name").str.to_lowercase().alias("tool_name"),
)

In [71]:
all_tool_df

name,output_file,parse_function_name,script_name,mamba_env,command,tool_name
str,str,str,str,str,str,str
"""blastn""","""{output_dir}/blastn_output.tsv""","""parse_blastn_custom""","""blastn.sh""","""blast_env""","""makeblastdb -in {contigs_file}…","""blastn"""
"""bowtie1""","""{output_dir}/bowtie1_output.sa…","""parse_sam""","""bowtie1.sh""","""bowtie_env""","""bowtie-build --threads {thread…","""bowtie1"""
"""bowtie2""","""{output_dir}/bowtie2_output.sa…","""parse_sam""","""bowtie2.sh""","""bowtie2_env""","""bowtie2-build --large-index --…","""bowtie2"""
"""indelfree_bruteforce""","""{output_dir}/indelfree_brutefo…","""parse_sam""","""indelfree_bruteforce.sh""","""bbmap_env""","""indelfree.sh subs=5 in={spacer…","""indelfree_bruteforce"""
"""indelfree_indexed""","""{output_dir}/indelfree_indexed…","""parse_sam""","""indelfree_indexed.sh""","""bbmap_env""","""indelfree.sh subs=5 in={spacer…","""indelfree_indexed"""
…,…,…,…,…,…,…
"""mmseqs2""","""{output_dir}/mmseqs_output.tsv""","""parse_blastn_custom""","""mmseqs.sh""","""mmseqs2_env""","""mkdir -p {output_dir}/tmp_spac…","""mmseqs2"""
"""mummer4""","""{output_dir}/mummer4_output.sa…","""parse_sam""","""mummer4.sh""","""mummer4_env""","""nucmer --maxmatch --nosimplify…","""mummer4"""
"""sassy""","""{output_dir}/sassy.tsv""","""parse_sassy""","""sassy.sh""","""benchy""","""sassy search --alphabet iupac …","""sassy"""


In [72]:
import subprocess
versions = {}
for tool in all_tool_df.iter_rows():
    # for each tool, try to try first word from the "command" 
    cmd = tool[5].split()[0]
    cmd_path = f"~/micromamba/envs/{tool[4]}/bin/{cmd} --version"
    try:
        result = subprocess.run([cmd_path], capture_output=True, text=True, shell=True, timeout=5)
        if result.returncode != 0:
            cmd_path = f"~/micromamba/envs/{tool[4]}/bin/{cmd} -version"
            result = subprocess.run([cmd_path], capture_output=True, text=True, shell=True, timeout=5)
            if result.returncode != 0:
                ix = tool[5].find("&&")
                if ix != -1:
                    for  part in tool[5].split("&&"):
                        cmd = part.strip().split()[0]
                        cmd_path = f"~/micromamba/envs/{tool[4]}/bin/{cmd} --version"
                        result = subprocess.run([cmd_path], capture_output=True, text=True, shell=True, timeout=5)
                        if result.returncode == 0:
                            break
                        cmd_path = f"~/micromamba/envs/{tool[4]}/bin/{cmd} -version"
                        result = subprocess.run([cmd_path], capture_output=True, text=True, shell=True, timeout=5)
                        if result.returncode == 0:
                            break
                        cmd_path = f"~/micromamba/envs/{tool[4]}/bin/{cmd} version"
                        result = subprocess.run([cmd_path], capture_output=True, text=True, shell=True, timeout=5)
                        if result.returncode == 0:
                            break
        # if result.returncode != 0:  
        print(cmd_path, result.stderr) #result.stdout
        versions[tool[0]] = result.stderr.strip() if result.stderr else result.stdout.strip()
    except Exception as e:
        versions[tool[0]] = f"Error: {e}"

~/micromamba/envs/blast_env/bin/makeblastdb -version 
~/micromamba/envs/bowtie_env/bin/bowtie-build --version 
~/micromamba/envs/bowtie2_env/bin/bowtie2-build --version 
~/micromamba/envs/bbmap_env/bin/indelfree.sh --version java -ea -Xmx185507m -Xms185507m -cp /clusterfs/jgi/groups/science/homes/uneri/micromamba/envs/bbmap_env/opt/bbmap-39.37-0/current/ aligner.IndelFreeAligner --version
BBTools version 39.37
For help, please run the shellscript with no parameters, or look in /docs/.

~/micromamba/envs/bbmap_env/bin/indelfree.sh --version java -ea -Xmx185535m -Xms185535m -cp /clusterfs/jgi/groups/science/homes/uneri/micromamba/envs/bbmap_env/opt/bbmap-39.37-0/current/ aligner.IndelFreeAligner --version
BBTools version 39.37
For help, please run the shellscript with no parameters, or look in /docs/.

~/micromamba/envs/lexicmap_env/bin/lexicmap version 
~/micromamba/envs/minimap2_env/bin/minimap2 --version 
~/micromamba/envs/mmseqs2_env/bin/mmseqs version 
~/micromamba/envs/mummer4_env/

In [73]:
versions

{'blastn': 'makeblastdb: 2.16.0+\n Package: blast 2.16.0, build Nov 27 2024 09:40:57',
 'bowtie1': 'bowtie-build version 1.3.1\n64-bit\nBuilt on fv-az1778-96\n2024-12-10T17:52:09\nCompiler: gcc version 13.3.0 (conda-forge gcc 13.3.0-1) \nOptions: -O3 -Wl,--hash-style=both -DPOPCNT_CAPABILITY -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /clusterfs/jgi/groups/science/homes/uneri/micromamba/envs/bowtie_env/include -fdebug-prefix-map=/opt/conda/conda-bld/bowtie_1733853009584/work=/usr/local/src/conda/bowtie-1.3.1 -fdebug-prefix-map=/clusterfs/jgi/groups/science/homes/uneri/micromamba/envs/bowtie_env=/usr/local/src/conda-prefix  -fvisibility-inlines-hidden -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /clusterfs/jgi/groups/science/homes/uneri/micromamba/envs/bowtie_env/include -fdebug-prefix-map=/opt/conda/conda-bld

In [74]:

all_tool_df = all_tool_df.join(pl.DataFrame(list(versions.items()), schema=["tool_name", "version_info"]), on="tool_name", how="left")
all_tool_df.write_csv("../tool_configs/tool_versions.csv")
all_tool_df

/clusterfs/jgi/groups/science/homes/uneri/micromamba/envs/benchy/lib/python3.10/functools.py:889: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


name,output_file,parse_function_name,script_name,mamba_env,command,tool_name,version_info
str,str,str,str,str,str,str,str
"""blastn""","""{output_dir}/blastn_output.tsv""","""parse_blastn_custom""","""blastn.sh""","""blast_env""","""makeblastdb -in {contigs_file}…","""blastn""","""makeblastdb: 2.16.0+ Package:…"
"""bowtie1""","""{output_dir}/bowtie1_output.sa…","""parse_sam""","""bowtie1.sh""","""bowtie_env""","""bowtie-build --threads {thread…","""bowtie1""","""bowtie-build version 1.3.1 64-…"
"""bowtie2""","""{output_dir}/bowtie2_output.sa…","""parse_sam""","""bowtie2.sh""","""bowtie2_env""","""bowtie2-build --large-index --…","""bowtie2""","""/clusterfs/jgi/groups/science/…"
"""indelfree_bruteforce""","""{output_dir}/indelfree_brutefo…","""parse_sam""","""indelfree_bruteforce.sh""","""bbmap_env""","""indelfree.sh subs=5 in={spacer…","""indelfree_bruteforce""","""java -ea -Xmx185507m -Xms18550…"
"""indelfree_indexed""","""{output_dir}/indelfree_indexed…","""parse_sam""","""indelfree_indexed.sh""","""bbmap_env""","""indelfree.sh subs=5 in={spacer…","""indelfree_indexed""","""java -ea -Xmx185535m -Xms18553…"
…,…,…,…,…,…,…,…
"""mmseqs2""","""{output_dir}/mmseqs_output.tsv""","""parse_blastn_custom""","""mmseqs.sh""","""mmseqs2_env""","""mkdir -p {output_dir}/tmp_spac…","""mmseqs2""","""18.8cc5c"""
"""mummer4""","""{output_dir}/mummer4_output.sa…","""parse_sam""","""mummer4.sh""","""mummer4_env""","""nucmer --maxmatch --nosimplify…","""mummer4""","""4.0.0rc1"""
"""sassy""","""{output_dir}/sassy.tsv""","""parse_sassy""","""sassy.sh""","""benchy""","""sassy search --alphabet iupac …","""sassy""","""sassy 0.1.4"""
